In [3]:
import meshplot as mp

ModuleNotFoundError: No module named 'meshplot'

In [ ]:
import numpy as np
import trimesh

from trimesh import proximity, sample

In [ ]:
# Utils 

def barycentric_coords(tris, P):
    """
    Return the barycentric coordinates of a point with respect to a triangle.
    """
    v0 = tris[2] - tris[0]
    v1 = tris[1] - tris[0]
    v2 = P - tris[0]
    d00 = np.dot(v0, v0)
    d01 = np.dot(v0, v1)
    d11 = np.dot(v1, v1)
    d20 = np.dot(v2, v0)
    d21 = np.dot(v2, v1)
    invDenom = d00 * d11 - d01 * d01
    v = (d11 * d20 - d01 * d21) * invDenom
    w = (d00 * d21 - d01 * d20) * invDenom
    u = 1.0 - v - w
    return np.array([u, v, w])


In [ ]:
mesh_name = 'uneven_plane'
mesh = trimesh.load(f'./data/{mesh_name}.obj')
hair = trimesh.load(f'./data/{mesh_name}_fur.obj')

amount = 1

# sample point cloud
# child_roots = mesh.sample_points_poisson_disk(amount, 1.5)
child_roots, faces = sample.sample_surface(mesh, amount)
child_normals = mesh.face_normals[faces]

In [ ]:
strand_len = 5
strands = np.array(np.split(hair.vertices, hair.vertices.shape[0] // strand_len))

root_mesh = trimesh.Trimesh(vertices=strands[:,0])
parents_pos, parents_ids = root_mesh.kdtree.query(child_roots, k=3)

parent_roots = np.array(root_mesh.vertices[parents_ids])

In [ ]:
#interpolate between parent and child
bary = barycentric_coords(parent_roots[0], child_roots[0]
bary

In [ ]:
strands_start = strands[::,1::]
strands_end = strands[::,:-1:]

# render the trimesh with meshplot
p = mp.plot(mesh.vertices, mesh.faces)
p.add_points(child_roots, c='red', shading=dict(point_size=.2))

p.add_points(strands[:,0], c='green', shading=dict(point_size=.05))
p.add_points(np.concatenate(parent_roots), c='blue', shading=dict(point_size=.2))

p.add_lines(np.concatenate(strands_start), 
            np.concatenate(strands_end), 
            shading=dict(line_width=.005, line_color='green'))

p.add_lines(child_roots,
            child_roots + child_normals,
            shading=dict(line_width=.05, line_color='red'))